In [37]:
!sudo pip install nltk prettytable openpyxl

In [2]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
# !gzip -d cc.ru.300.bin.gz
# !pip install -U bitsandbytes
# !pip install fasttext faiss-cpu nltk transformers sentencepiece langchain_community

import pandas as pd
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
# !gzip -d cc.ru.300.bin.gz
# !pip install fasttext faiss-cpu nltk transformers sentencepiece
import pandas as pd
import warnings
import re
# import fasttext
import faiss
# import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import string
import nltk
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader

import os

from tqdm.notebook import tqdm

from prettytable import PrettyTable
import random
import numpy as np
import gc
import scipy.io as sio
import time
import os
import pandas as pd
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
torch.manual_seed(42)
import time
import warnings
warnings.simplefilter('ignore')
import re
from transformers import logging
logging.set_verbosity_error()
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef

from torchvision import datasets, transforms as T

from transformers import AutoTokenizer, AutoModel, BertModel

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
from nltk.corpus import stopwords as nl_stopwords
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from sklearn.metrics.pairwise import linear_kernel
from scipy.spatial.distance import cosine
warnings.filterwarnings('ignore')
import warnings
import re
# import fasttext
import faiss
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import string
import nltk
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader

import os

from tqdm.notebook import tqdm

from prettytable import PrettyTable
import random
import numpy as np
import gc
import scipy.io as sio
import time

from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef

from torchvision import datasets, transforms as T

from transformers import AutoTokenizer, AutoModel, BertModel

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
from nltk.corpus import stopwords as nl_stopwords
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from sklearn.metrics.pairwise import linear_kernel
from scipy.spatial.distance import cosine
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/user1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/user1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [4]:
def clear_cache():
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    torch.backends.cudnn.deterministic=True

In [5]:
stopwords = nl_stopwords.words('russian')

##  vvv


In [6]:
!ls ./data/05_Таблица классификаторов.xlsx

ls: cannot access './data/05_Таблица': No such file or directory
ls: cannot access 'классификаторов.xlsx': No such file or directory


In [7]:
baza_df = pd.read_excel("./data/01_База_знаний.xlsx")
df = pd.read_excel('./data/02_Реальные_кейсы.xlsx')
df_classes = pd.read_excel('./data/05_Таблица классификаторов.xlsx')
baza_df.head()

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [8]:
p_d = './,][-")(~!#@^%$;*?&№∙^:<:>=_+\|`1°234}{567890'

def preprocess(text):
    output = text.replace('\n', ' ').replace('\t', ' ').replace('\u200c', ' ')
    output = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "", output)
    output = re.sub(r'^https?:\/\/.*[\r\n]*', '', output, flags=re.MULTILINE)
    for k in p_d:
        output = output.replace(k,' ')
    output = output.replace('  ', ' ')
    return output.strip()

**PREPROCESS_DATA**

In [9]:
df = pd.read_csv('./data/train_normal.csv')

In [10]:
df['additional'] = ' | ' + df["top_5"]
for i in range(5):
    df['additional'] += ' | ' + df[f"top_{i}"]


In [11]:
train_data = df[['question', 'additional', 'class_1', 'class_2']]

In [12]:
classes_1 = train_data['class_1'].unique()
classes_1_str2int = {classes_1[i]:i for i in range(len(classes_1))}
classes_1_int2str = {i:classes_1[i] for i in range(len(classes_1))}
classes_2 = train_data['class_2'].unique()
classes_2_str2int = { classes_2[i]:i for i in range(len(classes_2))}
classes_2_int2str = { i:classes_2[i] for i in range(len(classes_2))}

In [13]:
train_data['y1'] = train_data['class_1'].apply(lambda x: classes_1_str2int[x])
train_data['y2'] = train_data['class_2'].apply(lambda x: classes_2_str2int[x])
train_data = train_data.drop(['class_1', "class_2"], axis=1)

In [14]:
train_data = train_data[~(train_data.question.apply(lambda x: len(str(x))) == 0)]

In [15]:
from random import randint

In [16]:
def aug_crop_lst(lst):
    
    p = randint(1,10)

    if p>7:
        k = randint(0,len(lst)-1)
        lst[k] = " "
    return ' '.join(lst)

In [17]:
def aug_crop_str(s):
    
    p = randint(1,100)
    if p>85:
        new_s = ''
        k = randint(0,len(s)-1)
        for i in range(len(s)):
            if i!= k:
                new_s += s[i]
    return s

In [18]:
# print(aug_crop_str('qaewsrtfgyhu hbjgvcf f ff yta'))

In [19]:
class ClassificationDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.text = data.question.to_list()
        self.additional = data.additional.to_list()
        self.target = data.y1.to_list()

    def __getitem__(self, idx):
        text, target = self.text[idx],self.target[idx]
        text = [aug_crop_str(item) for item in text.split()]
        try:
            text = aug_crop_lst(text)
        except:
            print('niga', text)
        return text +' ' + self.additional[idx], target

    def __len__(self):
        return len(self.text)

def collate_fn(batch):
    model_input = []
    model_target = []
    for text, target in batch:
        model_input.append(text)
        model_target.append(target)

    tok = tokenizer(model_input, padding=True,
                    max_length=512, truncation=True,
                    return_tensors='pt')
    return tok, torch.tensor(model_target).long()


In [20]:
def get_loader(dataset, shuffle, batch_size):
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=False,
        num_workers=0,
        collate_fn=collate_fn
    )
    return loader


In [21]:
train_data['origin'] = 'sint'
train_data.loc[:801, 'origin'] = 'original'

In [22]:
for_split_data = train_data[train_data['origin'] == 'original'].dropna()

In [23]:
for_split_data

,question,additional,y1,y2,origin
0,Здравствуйте Можно уточнить причины Правилhttp...,| Что нельзя публиковать на RUTUBE | Что нель...,0,0,original
1,Добрый вечер какой топ причин блокировки видео...,| Общие вопросы | Не могу посмотреть ролик в ...,0,0,original
2,Все пишут что монетизация на рутубе отключаетс...,| Почему могут отключить монетизацию на видео...,1,1,original
3,Что запрещено в монетизации и что можно выклад...,| Почему могут отключить монетизацию на видео...,1,1,original
4,Чтобы не отключали монетизацию надо чтобы я са...,| Почему могут отключить монетизацию на видео...,1,1,original
...,...,...,...,...,...
796,Добрый день Как добавить таймкоды в видео,| Загрузка видео | Как добавить тайм коды под...,6,12,original
797,Как добавить временные интервалы,| Загрузка видео | Как добавить тайм коды под...,6,12,original
798,В каком формате добавляются таймкоды,| Загрузка видео | В каком формате нужно доба...,6,12,original
799,Как мне правильно указать тайм код,| Загрузка видео | В каком формате нужно доба...,6,12,original


In [24]:
X_train, X_test = train_test_split(for_split_data.drop(['y2'], axis=1), test_size=0.1, random_state=42, stratify=for_split_data['y1'])
# train_data_1, valid_df_1 = train_test_split(train_data_1, test_size=0.1, random_state=42, stratify=train_data_1['y1'])

In [25]:
X_train

,question,additional,y1,origin
340,А стримить это как,| Общие вопросы | Что такое трансляции стримы...,7,original
583,Есть возвращение пароля,| Общие вопросы | Как восстановить пароль | Ч...,2,original
758,Как скачать видео,| Общие вопросы | Как загрузить видеоролик на...,4,original
600,Удалил свой аккаунт восстановите плих,| Общие вопросы | Я удалил свой аккаунт а теп...,2,original
129,Как выйти из акаунта в приложении студии,| Мобильное приложение Студия RUTUBE на Andro...,2,original
...,...,...,...,...
494,Мне не подтвердили галочку что теперь,| Верификация | Что делать если заявку не одо...,2,original
573,Зачем мне регистрация,| Общие вопросы | Зачем нужна регистрация на ...,2,original
205,Можно дать профилю роль редактора с ограниченн...,| Предоставление доступа к каналу в Студии RU...,5,original
69,уже можно настраивать доступп к студии через п...,| Мобильное приложение Студия RUTUBE на IOS |...,5,original


In [26]:
external_data = train_data[train_data['origin'] == 'sint'].dropna().drop(columns=['y2'])

In [27]:
X_train = pd.concat([X_train, external_data])

In [28]:
class BertCLS(nn.Module):
    def __init__(self, model, n_classes):
        super(BertCLS, self).__init__()
        self.model = model
        self.fc = nn.Linear(1024, n_classes)
    
    def forward(self, batch):
        return self.fc(self.model(**batch).pooler_output)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "deepvk/USER-bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)


model = BertModel.from_pretrained(
            model_name, 
            ignore_mismatched_sizes=True, 
            num_labels=len(classes_2)
        )
bert_cls = BertCLS(model, n_classes=len(classes_2))

In [29]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

In [30]:
set_seed(42)
# device = 'cpu'
bert_cls = bert_cls.to(device)

num_epochs = 10
gradient_accumulation_steps = 1  # Increase to simulate a larger batch size

batch_size = 32
train_dataset = ClassificationDataset(X_train)
train_loader = get_loader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = ClassificationDataset(X_test)
test_loader = get_loader(test_dataset, shuffle=False, batch_size=batch_size)

# Total training steps calculation
total_steps = len(train_loader) * num_epochs // gradient_accumulation_steps

# Optimizer with a smaller learning rate and weight decay for regularization
optimizer = optim.AdamW(bert_cls.parameters(), lr=3e-5, weight_decay=0.01)

scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-5,
                                          total_steps=total_steps,
                                          div_factor=25,
                                          pct_start=0.1)
loss_func = torch.nn.CrossEntropyLoss()

In [31]:
def train(model, loss_func, device, train_loader, optimizer, epoch, gradient_accumulation_steps, scheduler):
    model.train()
    train_loader_length = len(train_loader) - 1 # хотим знать когда эпоха закончится
    pbar = tqdm(train_loader)
    for batch_idx, (data, labels) in enumerate(pbar):
        data, labels = data.to(device), labels.to(device)
        
        t = time.time()
        embeddings = model(data)
        t2 = time.time() - t
        
        loss = loss_func(embeddings, labels) / gradient_accumulation_steps
        
#         tot_m, used_m, free_m = map(int, os.popen('free -t -m').readlines()[-1].split()[1:])
        pbar.set_description(f"Loss Train: {float(loss) * gradient_accumulation_steps} Time: {t2}")
        
        loss.backward()
        
        if (batch_idx % gradient_accumulation_steps == 0) or (batch_idx == train_loader_length):
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1) # ставим модель в рамки, помогает
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
        
        del data, labels, embeddings, loss
    return model


In [32]:
def test(model, loader, device):
    y_true = []
    pred = []
    model.eval()
    with torch.no_grad():
        pbar = tqdm(loader) 
        for batch_idx, (data, labels) in enumerate(pbar):
            data = data.to(device)
            embeddings = model(data)
            pred.extend(embeddings.argmax(-1).detach().cpu().numpy())
            y_true.extend(labels.detach().cpu().numpy())

    return y_true, pred

In [33]:
def metrics(y_true, pred):
    print("matthews_corrcoef:", matthews_corrcoef(y_true, pred))
    print("accuracy_score:", accuracy_score(y_true, pred))
    print("f1_score:", f1_score(y_true, pred, average='weighted'))

In [34]:
for x in bert_cls.parameters(): x.requires_grad = True

In [35]:
# count_parameters(bert_cls)

In [36]:
# !mkdir 'models-all-classes'
for epoch in range(1, num_epochs + 1):
    bert_cls.train()
    
    clear_cache()
    train(bert_cls, loss_func, device, train_loader, optimizer,
          epoch, gradient_accumulation_steps, scheduler)
    PATH = f"./models-all-classes/{bert_cls.__class__.__name__}_epoch_{epoch}.pth"
    
    
    clear_cache()
    y_true, pred = test(bert_cls, test_loader, device)
    metrics(y_true, pred)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

matthews_corrcoef: 0.5813169861250697
accuracy_score: 0.654320987654321
f1_score: 0.6118933307204912


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

matthews_corrcoef: 0.8395478722173958
accuracy_score: 0.8641975308641975
f1_score: 0.8614085129080725


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

matthews_corrcoef: 0.8685062694692066
accuracy_score: 0.8888888888888888
f1_score: 0.8850186165000979


In [37]:
torch.save(bert_cls, PATH)